In [2]:
# Dataset Source:
# This dataset was collected from publicly available knitting and crochet pattern pages on Ravelry.
# Official website: https://www.ravelry.com
# The collected metadata includes pattern titles, designer names, descriptions, fiber types, skill levels, and tags.
# The data is used exclusively for research purposes to enhance pattern search functionality using NLP.

!pip install selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

#Extracts pattern URLs from Ravelry given a list of search queries and set number of results pages from which to scrape URLs
#Ravelry login credentials
USERNAME = "rlop373"
PASSWORD = "C@pstonec964"

# Set Up ChromeDriver
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)

chromedriver_path = r"C:\Users\raely\Downloads\chromedriver-win64 (1)\chromedriver-win64\chromedriver.exe"
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)
    
# Function to log in to Ravelry
def login_to_ravelry():
    driver.get("https://www.ravelry.com/account/login")
    time.sleep(3)

    # Find username & password fields
    username_field = driver.find_element(By.ID, "user_login")
    password_field = driver.find_element(By.ID, "user_password")
    
    # Enter credentials
    username_field.send_keys(USERNAME)
    password_field.send_keys(PASSWORD)
    password_field.send_keys(Keys.RETURN)

    time.sleep(5)  # Wait for login to process
    print("Logged in to Ravelry successfully.")

# Function to Scrape Pattern URLs (Removes Duplicates)
def scrape_pattern_urls(query, max_pages=1, delay=3):
    pattern_urls = set()
    
    for page in range(1, max_pages + 1):
        search_url = f"https://www.ravelry.com/patterns/search#query={query}&sort=popularity&view=captioned_thumbs&page={page}"
        driver.get(search_url)
        time.sleep(delay)

        # Find pattern links using XPath
        links = driver.find_elements(By.XPATH, "//a[contains(@href, '/patterns/library/')]")

        for link in links:
            pattern_urls.add(link.get_attribute("href"))

        print(f"{query}: Page {page} scraped. Found {len(pattern_urls)} unique patterns so far.")

    return list(pattern_urls)

def extract_urls():
    # Queries list to search
    queries = ["sweater", "shawl", "scarf", "cardigan", "hat", "mittens", "crochet", "vest", "tunisian", "cowl"]
    login_to_ravelry()
    
    # Loop through all queries and scrape URLs
    all_pattern_urls = []
    for query in queries:
        pattern_urls = scrape_pattern_urls(query)
        all_pattern_urls.extend(pattern_urls)
    
    # Save to CSV (also removes any duplicates in pattern entries)
    df_patterns = pd.DataFrame({"URL": list(set(all_pattern_urls))})
    df_patterns.to_csv("ravelry_pattern_urls.csv", index=False)
    
    print(f"Scraping complete. {len(set(all_pattern_urls))} unique pattern URLs saved to ravelry_pattern_urls.csv")
    
    # Close the WebDriver
    driver.quit()


Logged in to Ravelry successfully.
sweater: Page 1 scraped. Found 48 unique patterns so far.
sweater: Page 2 scraped. Found 96 unique patterns so far.


KeyboardInterrupt: 